In [42]:
import os
from time import sleep
import json
import argparse
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.prompts import PromptTemplate
import base64

In [43]:
parser = argparse.ArgumentParser(description="test LLM planning abilities")
parser.add_argument(
        "--dim",
        type=int,
        default=1,
        help="dimension of the problem 1 or 2 (for 1D or 2D)",
)
parser.add_argument(
        "--prompt_type",
        type=str,
        default='text',
        help="type of prompt: text, image, or both",
    )
parser.add_argument(
        "--color",
        type=int,
        default=0,
        help="Use color images (1) or not (0)",
    )
parser.add_argument(
        "--model_name",
        type=str,
        default='all',
        help="num",
    )
parser.add_argument(
        "--openai_key",
        type=str,
        default='',
        help="num",
    )
parser.add_argument(
        "--gemini_key",
        type=str,
        default='',
        help="num",
    )

args_string = '--dim=1 --prompt_type=text --color=0'
args_list = args_string.split(' ')
args = parser.parse_args(args_list)

In [44]:
if args.model_name=='all':
    model_name = [
        'gpt-3.5-turbo',
        'gpt-4o-mini',
        'gpt-4o-2024-08-06'
    ]
else:
    model_name = args.model_name
# model_name='gpt-3.5-turbo'
model_name='gpt-4o-mini'
# model_name='gpt-4o-2024-08-06'
# model_name='gemini-1.5-pro'

if not args.openai_key:
    apikey_filepath = '../.openai_key.txt'
    with open(apikey_filepath, 'r') as f:
        openai_key = f.read()
else:
    openai_key = args.key

if not args.gemini_key:
    gemini_key_filepath = '../.gemini_api_key.txt'
    with open(gemini_key_filepath, 'r') as f:
        gemini_key = f.read()
else:
    gemini_key = args.gemini_key

if 'gpt' in model_name:
    print('gpt model')
    client = OpenAI(api_key=openai_key, model_name=model_name)
elif 'gemini' in model_name:
    print('gemini model')
    client = ChatGoogleGenerativeAI(google_api_key=gemini_key, model=model_name)

gpt model


In [45]:
# Load Data
if args.dim == 1:
    examples_data_dir = '../data/brick_1D_50'
elif args.dim == 2:
    examples_data_dir = '../data/brick_2D_50'

with open(os.path.join(examples_data_dir, 'data.json')) as f:
    data = json.load(f)

zeroshot_prompt = 'Lets think step by step, and provide the answer in the format of a sequence of bricks by a comma in the last sentence.'

In [49]:
if args.prompt_type.lower() in ['text', 'txt']:
    from openai import OpenAI
    # generate image based on the text prompt
    res_list = []

    text_prompt_prefix = 'Given the textual description below, generate an image that explains to another instance of AI what this text describes, so that you can both answer spatial reasoning questions about it in the same way.' 
    prompt = PromptTemplate(
        input_variables=["image_desc"],
        template= text_prompt_prefix + "\n\n {image_desc}",
    )
    chain = LLMChain(llm=llm, prompt=prompt)

    for i, item in enumerate([data[0]]):
        print('step: ', i+1)

        client = OpenAI(api_key=openai_key)

        response = client.images.generate(
            model="dall-e-3",
            prompt=text_prompt_prefix + "\n\n {item['data']}",
            size="1024x1024",
            quality="standard",
            n=1,
            )

        image_url = response.data[0].url

        print(image_url)
        # res = response.content

        # dict_res = {'pred': res, 'label': item['label']}
        # res_list.append(dict_res)



ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [52]:
text_prompt_prefix + f"\n\n {item['data']}"

'Given the textual description below, generate an image that explains to another instance of AI what this text describes, so that you can both answer spatial reasoning questions about it in the same way.\n\n There is a set of bricks. The brick V is on top of the brick N . The brick S is on top of the brick V . The brick F is on top of the brick E . For the brick J, the color is blue. The brick H is on top of the brick S . The brick R is on top of the brick M . The brick M is on top of the brick I . The brick I is on top of the brick H . The brick N is on top of the brick F . The brick E is on top of the brick J . Now we have to get a specific brick. The bricks must now be grabbed from top to bottom, and if the lower brick is to be grabbed, the upper brick must be removed first. How to get brick J?'

In [50]:
if args.prompt_type.lower() in ['text', 'txt']:
    # generate image based on the text prompt
    res_list = []
    llm = OpenAI(api_key=openai_key)

    text_prompt_prefix = 'Given the textual description below, generate an image that explains to another instance of AI what this text describes, so that you can both answer spatial reasoning questions about it in the same way.' 
    prompt = PromptTemplate(
        input_variables=["image_desc"],
        template= text_prompt_prefix + "\n\n {image_desc}",
    )
    chain = LLMChain(llm=llm, prompt=prompt)

    for i, item in enumerate([data[0]]):
        print('step: ', i+1)

        image_url = DallEAPIWrapper(api_key=openai_key).run(chain.run(item['data']))
        print(image_url)
        # res = response.content

        # dict_res = {'pred': res, 'label': item['label']}
        # res_list.append(dict_res)


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [ ]:
res_list

[{'pred': 'To visualize the arrangement of the bricks based on the description provided, we can create a diagram that shows the stacking of the bricks. Here’s how the bricks are arranged according to the description:\n\n1. **Top Layer**:\n   - H (on top of S)\n   - S (on top of V)\n   - V (on top of N)\n\n2. **Middle Layer**:\n   - N (on top of F)\n   - F (on top of E)\n   - E (on top of J)\n\n3. **Bottom Layer**:\n   - R (on top of M)\n   - M (on top of I)\n   - I (on top of H)\n\nNow, to get brick J, we need to follow the order of removal from top to bottom:\n\n1. Remove brick H (which is on top of I).\n2. Remove brick I (which is on top of M).\n3. Remove brick M (which is on top of R).\n4. Remove brick R.\n5. Remove brick N (which is on top of F).\n6. Remove brick F (which is on top of E).\n7. Finally, remove brick E (which is on top of J).\n8. Now, brick J can be taken.\n\nThis sequence of removals ensures that we can access brick J by following the stacking order. \n\nThe image sh

In [30]:
if args.prompt_type.lower() in ['image','img']:

    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    

    image_subdir = 'images'
    if args.color:
        color='color'
    else:
        color='bw'

    image_prompt_prefix = 'Given the image below, explain to another instance of AI what this image shows, so that you can both answer spatial reasoning questions about it in the same way.'
    # img_prompt = 'The image shows a set of bricks that can be placed on top of each other. Now we have to get a specific brick. The bricks must now be grabbed from top to bottom, and if the lower brick is to be grabbed, the upper brick must be removed first. How to get brick {t}?'
    res_list = []
    i = 0

    for i, item in enumerate(data):
        print('step: ', i+1)
        image_path = os.path.join(examples_data_dir, image_subdir, f'img_{i}_{color}.png')
        image_base64 = encode_image(image_path)

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", image_prompt_prefix),
                (
                    "user",
                    [
                        {
                            "type": "image_url",
                            "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                        }
                    ],
                ),
            ]
        )
        chain = prompt | client
        res = chain.invoke({'image_data':image_base64}).content
        dict_res = {'pred': res, 'label': item['label']}
        res_list.append(dict_res)

In [31]:
if args.prompt_type.lower() in ['both','all','text+img']:

    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    

    image_subdir = 'images'
    if args.color:
        color='color'
    else:
        color='bw'

    txt_img_prompt_suffix = ' The described brick layout is visualized in the attached image. You can use both image and textual description to solve the task.'
    res_list = []
    i = 0

    for i, item in enumerate(data):
        print('step: ', i+1)
        image_path = os.path.join(examples_data_dir, image_subdir, f'img_{i}_{color}.png')
        image_base64 = encode_image(image_path)

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", zeroshot_prompt),
                (
                    "user",
                    [
                        {
                            "type": "text",
                            "text": item['data'] + txt_img_prompt_suffix
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                        }
                    ],
                ),
            ]
        )
        chain = prompt | client
        res = chain.invoke({'image_data':image_base64}).content
        dict_res = {'pred': res, 'label': item['label']}
        res_list.append(dict_res)

step:  1
step:  2
step:  3
step:  4
step:  5
step:  6
step:  7
step:  8
step:  9
step:  10
step:  11
step:  12
step:  13
step:  14
step:  15
step:  16
step:  17
step:  18
step:  19
step:  20
step:  21
step:  22
step:  23
step:  24
step:  25
step:  26
step:  27
step:  28
step:  29
step:  30
step:  31
step:  32
step:  33
step:  34
step:  35
step:  36
step:  37
step:  38
step:  39
step:  40
step:  41
step:  42
step:  43
step:  44
step:  45
step:  46
step:  47
step:  48
step:  49
step:  50


In [32]:
if args.prompt_type.lower() in ['img','image','both','all','text+img']:
    colorname = '_color' if args.color else '_bw'
else:
    colorname = ''

output_path = os.path.join('../data/results/', f'{args.dim}D_{args.prompt_type}{colorname}_{model_name}.json')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w') as outfile:
    json.dump(res_list, outfile)

print(f'Results saved to {output_path}')

Results saved to ../data/results/1D_both_color_gpt-4o-2024-08-06.json
